In [1]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import SimpleITK as sitk
import shutil
from scipy.ndimage import binary_erosion, distance_transform_edt
import Aorta_segmentation_ZC.functions_collection as ff
import Aorta_segmentation_ZC.Data_processing as Data_processing

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
patient_list = pd.read_excel('/host/d/Data/CTA/Patient_lists/resampled_data_info_with_train_test_split_TAA.xlsx')
patient_list = patient_list[patient_list['batch']=='test']
print(patient_list.shape)

save_path = '/host/d/projects/aorta_seg/models/Dataset504_AortaTAA/results/EncUNetM_3d_fullres/predicts'



(13, 13)


In [46]:
fold = 0
for idx in range(patient_list.shape[0]):
    patient_class = patient_list.iloc[idx]['class']
    patient_subclass = str(patient_list.iloc[idx]['subclass'])
    patient_id = patient_list.iloc[idx]['id']
    patient_index = patient_list.iloc[idx]['patient_index']

    save_folder = os.path.join(save_path,  patient_class, patient_subclass, patient_id, 'fold_'+str(fold))
    ff.make_folder([os.path.join(save_path, patient_class),
                    os.path.join(save_path, patient_class, patient_subclass),
                    os.path.join(save_path, patient_class, patient_subclass, patient_id),
                    save_folder])
    
    # original path filename is Aorta_XXXX where XXXX is the patient_index with leading zeros
    original_path = os.path.join(os.path.dirname(save_path), 'predicts_raw/fold_'+str(fold), 'AortaTAA_' + str(patient_index).zfill(4) + '.nii.gz')
    save_filename = os.path.join(save_folder,'pred_seg.nii.gz')

    shutil.copyfile(original_path, save_filename)



### ensemble folds

In [55]:
patient_list = pd.read_excel('/host/d/Data/CTA/Patient_lists/resampled_data_info_with_train_test_split_TAA.xlsx')
patient_list = patient_list[patient_list['batch']=='test']
print(patient_list.shape)

save_path = '/host/d/projects/aorta_seg/models/Dataset504_AortaTAA/results/EncUNetM_3d_fullres/predicts'

(13, 13)


In [53]:
import numpy as np
from scipy import ndimage

def remove_scatter(mask3d: np.ndarray,
                   target_label: int = 1,
                   connectivity: int = 26,
                   return_is_single: bool = False):
    """
    Keep only the largest 3D connected component of voxels == target_label.

    Args:
        mask3d: 3D numpy array, e.g. (X,Y,Z) with integer labels.
        target_label: which label to process (default 1).
        connectivity: 6, 18, or 26 for 3D connectivity.
        return_is_single: if True, also return whether it was a single component.

    Returns:
        cleaned_mask: same shape as mask3d, only largest component kept for target_label.
        (optional) is_single: bool, whether original target_label voxels form exactly 1 component.
        (optional) n_components: int, number of connected components (excluding background).
    """
    assert mask3d.ndim == 3, f"mask3d must be 3D, got {mask3d.ndim}D"

    # binary mask for the target label
    bin_mask = (mask3d == target_label)

    # if empty, return as-is
    if not np.any(bin_mask):
        if return_is_single:
            return mask3d.copy(), True, 0
        return mask3d.copy()

    # choose connectivity structure
    if connectivity == 6:
        structure = ndimage.generate_binary_structure(3, 1)
    elif connectivity == 18:
        structure = ndimage.generate_binary_structure(3, 2)
        # generate_binary_structure(3,2) actually gives 18-neighborhood in 3D
    elif connectivity == 26:
        structure = np.ones((3, 3, 3), dtype=bool)
    else:
        raise ValueError("connectivity must be one of {6, 18, 26}")

    labeled, ncomp = ndimage.label(bin_mask, structure=structure)

    # 判断是否单一连通域
    is_single = (ncomp == 1)

    if ncomp <= 1:
        cleaned = mask3d.copy()
        if return_is_single:
            return cleaned, is_single, ncomp
        return cleaned

    # compute component sizes (exclude background label 0)
    sizes = ndimage.sum(bin_mask, labeled, index=np.arange(1, ncomp + 1))
    largest_cc = int(np.argmax(sizes) + 1)

    largest_mask = (labeled == largest_cc)

    # build output: keep only largest CC for target_label, keep other labels unchanged
    cleaned = mask3d.copy()
    # remove all target_label first
    cleaned[bin_mask] = 0
    # put back the largest CC as target_label
    cleaned[largest_mask] = target_label

    if return_is_single:
        return cleaned, is_single, ncomp
    return cleaned


In [ ]:
for idx in range(patient_list.shape[0]):
    patient_class = patient_list.iloc[idx]['class']
    patient_subclass = str(patient_list.iloc[idx]['subclass'])
    patient_id = patient_list.iloc[idx]['id']
    patient_index = patient_list.iloc[idx]['patient_index']

    # print('Processing patient:', patient_id)

    save_folder = os.path.join(save_path,  patient_class, patient_subclass, patient_id, 'final')
    ff.make_folder([save_folder])
    
    fold0_result = os.path.join(os.path.dirname(save_folder), 'fold_0/pred_seg.nii.gz')
    affine = nb.load(fold0_result).affine
    fold0_result = nb.load(fold0_result).get_fdata()

    # fold1_result = os.path.join(os.path.dirname(save_folder), 'fold_1/pred_seg.nii.gz')
    # fold1_result = nb.load(fold1_result).get_fdata()
    # fold2_result = os.path.join(os.path.dirname(save_folder), 'fold_2/pred_seg.nii.gz')
    # fold2_result = nb.load(fold2_result).get_fdata()

    # stacked = np.stack([fold0_result, fold1_result, fold2_result], axis=0)
    # final_result = (np.sum(stacked, axis=0) >= 2).astype(np.uint8)

    final_result = fold0_result

    # do connected component analysis to keep the largest component only
    final_result, is_single ,_ = remove_scatter(final_result, target_label=1, connectivity=26, return_is_single=True)
    # print('is single component before cleaning:', is_single)
   

    nb.save(nb.Nifti1Image(final_result, affine), os.path.join(save_folder, 'pred_seg.nii.gz'))



Processing patient: t0
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True
Processing patient: t0
is single component before cleaning: False
Processing patient: t1
is single component before cleaning: True
Processing patient: t0
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True
Processing patient: t0
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True
Processing patient: t0
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True
Processing patient: t0
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True
Processing patient: t1
is single component before cleaning: True


### resample back to original spacing

In [3]:
patient_list = pd.read_excel('/host/d/Data/CTA/Patient_lists/resampled_data_info_with_train_test_split_TAA.xlsx')
patient_list = patient_list[patient_list['batch']=='test']
print(patient_list.shape)

original_spacing_info = pd.read_excel('/host/d/Data/CTA/Patient_lists/original_data_info_TAA.xlsx', dtype={'subclass': str})

save_path = '/host/d/projects/aorta_seg/models/Dataset504_AortaTAA/results/EncUNetM_3d_fullres/predicts'


(13, 13)


In [21]:
for i in range(0, patient_list.shape[0]):
    patient_class = patient_list.iloc[i]['class']
    patient_subclass = str(patient_list.iloc[i]['subclass'])
    patient_id = patient_list.iloc[i]['id']
    patient_index = patient_list.iloc[i]['patient_index']
    print('patient subclass and id:', patient_subclass, patient_id)

    save_folder = os.path.join(save_path,  patient_class, patient_subclass, patient_id,'final')

    pred_path = os.path.join(save_folder, 'pred_seg.nii.gz')

    # find out original spacing
    # row is confirmed by maching patient_subclass and patient_id
    original_record = original_spacing_info[(original_spacing_info['subclass']==patient_subclass) & (original_spacing_info['id']==patient_id)]

    original_spacing_x = original_record['spacing_x'].values[0]
    original_spacing_y = original_record['spacing_y'].values[0]
    original_spacing_z = original_record['spacing_z'].values[0]

    print('original spacing:', original_spacing_x, original_spacing_y, original_spacing_z)

    # do segmentation
    seg_nifti = nb.load(pred_path)
    seg = seg_nifti.get_fdata()
    seg_resampled_nifti = Data_processing.resample_nifti(seg_nifti, order = 0, mode = 'nearest', cval = 0, in_plane_resolution_mm = original_spacing_x, slice_thickness_mm = original_spacing_z, number_of_slices = None)
    header = seg_resampled_nifti.header
    affine = seg_resampled_nifti.affine
    seg_resampled_data = seg_resampled_nifti.get_fdata()
    seg_resampled_data = np.round(seg_resampled_data).astype(np.int16)

    # check dimensions
    original_seg = os.path.join('/host/d/Data/CTA/original_data/', patient_class, patient_subclass, patient_id, 'seg_cropped.nii.gz')
    original_seg_nifti = nb.load(original_seg).get_fdata()

    if seg_resampled_data.shape != original_seg_nifti.shape:
        print('index i :', i, ' patient_subclass:', patient_subclass, ' patient_id:', patient_id, ' dimension mismatch before cropping:', seg_resampled_data.shape, original_seg_nifti.shape)
        seg_resampled_data = Data_processing.crop_or_pad(seg_resampled_data, original_seg_nifti.shape, 0)
        seg_resample_data = Data_processing.correct_shift_caused_in_pad_crop_loop(seg_resampled_data)
        print('After cropping/padding, new shape:', seg_resampled_data.shape)
        assert seg_resampled_data.shape == original_seg_nifti.shape, f"After cropping/padding, shape still mismatch: {seg_resampled_data.shape} vs {original_seg_nifti.shape}"

    seg_resampled_nifti = nb.Nifti1Image(seg_resampled_data, affine, header)
    nb.save(seg_resampled_nifti, os.path.join(save_folder, 'pred_seg_original_res.nii.gz'))

patient subclass and id: 218 t0
original spacing: 0.822265625 0.822265625 0.800000011920929
index i : 0  patient_subclass: 218  patient_id: t0  dimension mismatch before cropping: (250, 379, 379) (250, 380, 379)
After cropping/padding, new shape: (250, 380, 379)
patient subclass and id: 218 t1
original spacing: 0.6835939884185791 0.6835939884185791 0.625
index i : 1  patient_subclass: 218  patient_id: t1  dimension mismatch before cropping: (250, 380, 504) (250, 380, 505)
After cropping/padding, new shape: (250, 380, 505)
patient subclass and id: 430 t0
original spacing: 0.76171875 0.76171875 0.800000011920929
patient subclass and id: 430 t1
original spacing: 0.68359375 0.68359375 0.800000011920929
patient subclass and id: 585 t0
original spacing: 0.73828125 0.73828125 0.800000011920929
patient subclass and id: 585 t1
original spacing: 0.697265625 0.697265625 0.699999988079071
index i : 5  patient_subclass: 585  patient_id: t1  dimension mismatch before cropping: (250, 381, 431) (250, 

### quantitative evaluation

In [24]:
patient_list = pd.read_excel('/host/d/Data/CTA/Patient_lists/resampled_data_info_with_train_test_split_TAA.xlsx')
patient_list = patient_list[patient_list['batch']=='test']
print(patient_list.shape)

pred_path = '/host/d/projects/aorta_seg/models/Dataset504_AortaTAA/results/EncUNetM_3d_fullres/predicts'
gt_path = '/host/d/Data/CTA/processed_data/'
original_gt_path = '/host/d/Data/CTA/original_data/'

(13, 13)


In [23]:
def HD_95_numpy(pred,gt,spacing):
    pred = pred.astype(bool)
    gt = gt.astype(bool)

    # surface voxels
    pred_surf = pred ^ binary_erosion(pred)
    gt_surf = gt ^ binary_erosion(gt)

    dt_gt = distance_transform_edt(~gt_surf, sampling=spacing)
    dt_pred = distance_transform_edt(~pred_surf, sampling=spacing)

    d_pred_gt = dt_gt[pred_surf]
    d_gt_pred = dt_pred[gt_surf]

    hd95=max(np.percentile(d_pred_gt,95), np.percentile(d_gt_pred,95))

    return hd95

In [26]:
results = []
for idx in range(patient_list.shape[0]):
   patient_class = patient_list.iloc[idx]['class']
   patient_subclass = str(patient_list.iloc[idx]['subclass'])
   patient_id = patient_list.iloc[idx]['id']
   patient_index = patient_list.iloc[idx]['patient_index']
    
   pred_filename = os.path.join(pred_path,  patient_class, patient_subclass, patient_id, 'fold_0', 'pred_seg.nii.gz')
   gt_filename = os.path.join(gt_path, patient_class, patient_subclass, patient_id,'seg_resampled.nii.gz')

   pred_original_res_filename = os.path.join(pred_path,  patient_class, patient_subclass, patient_id, 'final', 'pred_seg_original_res.nii.gz')
   gt_original_res_filename = os.path.join(original_gt_path, patient_class, patient_subclass, patient_id,'seg_cropped.nii.gz')
   # pred_final_filename = os.path.join(pred_path,  patient_class, patient_subclass, patient_id, 'final', 'pred_seg.nii.gz')

   pred_seg = nb.load(pred_filename).get_fdata()
   pred_original_res = nb.load(pred_original_res_filename).get_fdata()
   # pred_seg_final = nb.load(pred_final_filename).get_fdata()

   gt_seg = nb.load(gt_filename)
   pixel_spacing = gt_seg.header.get_zooms()
   print('pixel spacing:', pixel_spacing)
   gt_seg = gt_seg.get_fdata()
  
   if np.unique(gt_seg).shape[0]>2:
      gt_seg[gt_seg>0] = 1  # convert to binary mask

   gt_original_res_seg = nb.load(gt_original_res_filename)
   pixel_spacing_original_res = gt_original_res_seg.header.get_zooms()
   gt_original_res_seg = gt_original_res_seg.get_fdata()
   if np.unique(gt_original_res_seg).shape[0]>2:
      gt_original_res_seg[gt_original_res_seg>0] = 1  # convert to binary mask

   dice_score = ff.np_categorical_dice(pred_seg, gt_seg, target_class=1)
   hd = HD_95_numpy(pred_seg, gt_seg, spacing=pixel_spacing)

   dice_score_original_res = ff.np_categorical_dice(pred_original_res, gt_original_res_seg, target_class=1)
   hd_original_res = HD_95_numpy(pred_original_res, gt_original_res_seg, spacing=pixel_spacing_original_res)

   # dice_score_final = ff.np_categorical_dice(pred_seg_final, gt_seg, target_class=1)
   # hd_final = HD_95_numpy(pred_seg_final, gt_seg, spacing=pixel_spacing)

   print('for patient_class: ', patient_class, ', subclass: ', patient_subclass, ', id: ', patient_id)
   print('Dice score: ', dice_score, ' original_res dice: ', dice_score_original_res)
   print('HD 95: ', hd, ' original_res hd: ', hd_original_res)

   results.append([patient_index, patient_class, patient_subclass, patient_id, dice_score, hd, dice_score_original_res, hd_original_res])
   df = pd.DataFrame(results, columns=['patient_index', 'class', 'subclass', 'id', 'dice_1.5mm', 'hd_1.5mm','dice_original_res', 'hd_original_res'])
   df.to_excel(os.path.join(os.path.dirname(pred_path), 'quantitative.xlsx'), index=False)


pixel spacing: (1.5, 1.5, 1.5009902)
for patient_class:  TAA_peking , subclass:  218 , id:  t0
Dice score:  0.964567077254158  original_res dice:  0.9565414498401236
HD 95:  2.1220206025104282  original_res hd:  2.294446144175214
pixel spacing: (1.5, 1.5, 1.5029762)
for patient_class:  TAA_peking , subclass:  218 , id:  t1
Dice score:  0.9657858333628939  original_res dice:  0.9583421444757664
HD 95:  2.1213203435596424  original_res hd:  1.9745895029119602
pixel spacing: (1.5, 1.5, 1.4995672)
for patient_class:  TAA_peking , subclass:  430 , id:  t0
Dice score:  0.9349328964392672  original_res dice:  0.9344297383157278
HD 95:  3.3541019662496847  original_res hd:  3.231698960891643
pixel spacing: (1.5, 1.5, 1.4991305)
for patient_class:  TAA_peking , subclass:  430 , id:  t1
Dice score:  0.9653288297238249  original_res dice:  0.956474284017235
HD 95:  2.1207055944609894  original_res hd:  2.2128041376247602
pixel spacing: (1.5, 1.5, 1.4985366)
for patient_class:  TAA_peking , subcla

### result presentation


In [32]:
patient_list = pd.read_excel('/host/d/Data/CTA/Patient_lists/resampled_data_info_with_train_test_split_TAA.xlsx')
patient_list = patient_list[patient_list['batch']=='test']
print(patient_list.shape)

pred_path = '/host/d/projects/aorta_seg/models/Dataset504_AortaTAA/results/EncUNetM_3d_fullres/predicts'
gt_original_path = '/host/d/Data/CTA/original_data/'
gt_processed_path = '/host/d/Data/CTA/processed_data/'

(13, 13)


In [34]:
results = []
for idx in range(0,patient_list.shape[0]):
   patient_class = patient_list.iloc[idx]['class']
   patient_subclass = str(patient_list.iloc[idx]['subclass'])
   patient_id = patient_list.iloc[idx]['id']
   patient_index = patient_list.iloc[idx]['patient_index']

   save_folder = os.path.join(os.path.dirname(pred_path), 'examples', patient_class, patient_subclass, patient_id)
   ff.make_folder([os.path.join(os.path.dirname(pred_path), 'examples', patient_class),
                   os.path.join(os.path.dirname(pred_path), 'examples', patient_class, patient_subclass),
                   os.path.join(os.path.dirname(pred_path), 'examples', patient_class, patient_subclass, patient_id),
                   save_folder])

   original_img_filename = os.path.join(gt_original_path, patient_class, patient_subclass, patient_id,'image_cropped.nii.gz')
   original_seg_filename = os.path.join(gt_original_path, patient_class, patient_subclass, patient_id,'seg_cropped.nii.gz')
   resampled_img_filename = os.path.join(gt_processed_path, patient_class, patient_subclass, patient_id,'img_resampled.nii.gz')
   resampled_seg_filename = os.path.join(gt_processed_path, patient_class, patient_subclass, patient_id,'seg_resampled.nii.gz')

   pred_seg_filename = os.path.join(pred_path,  patient_class, patient_subclass, patient_id, 'final', 'pred_seg.nii.gz')
   pred_seg_original_res_filename = os.path.join(pred_path,  patient_class, patient_subclass, patient_id, 'final', 'pred_seg_original_res.nii.gz')

   shutil.copyfile(original_img_filename, os.path.join(save_folder, 'image_cropped.nii.gz'))
   shutil.copyfile(original_seg_filename, os.path.join(save_folder, 'gt_seg_cropped.nii.gz'))
   shutil.copyfile(resampled_img_filename, os.path.join(save_folder, 'img_resampled_1.5mm.nii.gz'))
   shutil.copyfile(resampled_seg_filename, os.path.join(save_folder, 'gt_seg_resampled_1.5mm.nii.gz'))
   shutil.copyfile(pred_seg_original_res_filename, os.path.join(save_folder, 'pred_seg_original_res.nii.gz'))
   shutil.copyfile(pred_seg_filename, os.path.join(save_folder, 'pred_seg_1.5mm.nii.gz'))

In [ ]:
# from scipy.ndimage import binary_erosion, distance_transform_edt
# pred = pred_seg.astype(bool)
# pred_surf = pred ^ binary_erosion(pred)

# # plot pred and pred_surf side by side to check
# import matplotlib.pyplot as plt
# slice_idx = pred.shape[2]//2
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plt.imshow(pred[:,:,slice_idx], cmap='gray')
# plt.title('Predicted Segmentation')
# plt.subplot(1,2,2)
# plt.imshow(pred_surf[:,:,slice_idx], cmap='gray')
# plt.title('Predicted Surface')
# plt.show()
